In [13]:
!wget http://shannon.cs.illinois.edu/DenotationGraph/data/flickr30k-images.tar
!tar -xvf flickr30k-images.tar

!wget http://shannon.cs.illinois.edu/DenotationGraph/data/flickr30k.tar.gz
!tar -xvf flickr30k.tar.gz

Streaming output truncated to the last 5000 lines.
flickr30k-images/5363903246.jpg
flickr30k-images/5364849864.jpg
flickr30k-images/536495604.jpg
flickr30k-images/5365075444.jpg
flickr30k-images/536533294.jpg
flickr30k-images/536537638.jpg
flickr30k-images/5366770416.jpg
flickr30k-images/536718060.jpg
flickr30k-images/536721406.jpg
flickr30k-images/5367322642.jpg
flickr30k-images/536739751.jpg
flickr30k-images/536828916.jpg
flickr30k-images/536904709.jpg
flickr30k-images/5369771639.jpg
flickr30k-images/5370581509.jpg
flickr30k-images/537069478.jpg
flickr30k-images/5371109894.jpg
flickr30k-images/5371186400.jpg
flickr30k-images/5371247136.jpg
flickr30k-images/5371507831.jpg
flickr30k-images/5371586024.jpg
flickr30k-images/537222436.jpg
flickr30k-images/537225246.jpg
flickr30k-images/537230454.jpg
flickr30k-images/537359971.jpg
flickr30k-images/5373606679.jpg
flickr30k-images/5373649841.jpg
flickr30k-images/537390477.jpg
flickr30k-images/5374764034.jpg
flickr30k-images/537479916.jpg
flic

In [0]:
token = 'results_20130124.token'
captions = open(token, 'r').read().strip().split('\n')

In [0]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [0]:
def load_doc(filename):
  text = open(filename, 'r').read().strip().split('\n')
  return text

In [17]:
token = 'results_20130124.token'

doc = load_doc(token)
doc[:10]

['1000092795.jpg#0\tTwo young guys with shaggy hair look at their hands while hanging out in the yard .',
 '1000092795.jpg#1\tTwo young , White males are outside near many bushes .',
 '1000092795.jpg#2\tTwo men in green shirts are standing in a yard .',
 '1000092795.jpg#3\tA man in a blue shirt standing in a garden .',
 '1000092795.jpg#4\tTwo friends enjoy time spent together .',
 '10002456.jpg#0\tSeveral men in hard hats are operating a giant pulley system .',
 '10002456.jpg#1\tWorkers look down from up above on a piece of equipment .',
 '10002456.jpg#2\tTwo men working on a machine wearing hard hats .',
 '10002456.jpg#3\tFour men on top of a tall structure .',
 '10002456.jpg#4\tThree men on a large rig .']

In [0]:
token = 'results_20130124.token'
captions = open(token, 'r').read().strip().split('\n')

In [19]:
captions[:10]

['1000092795.jpg#0\tTwo young guys with shaggy hair look at their hands while hanging out in the yard .',
 '1000092795.jpg#1\tTwo young , White males are outside near many bushes .',
 '1000092795.jpg#2\tTwo men in green shirts are standing in a yard .',
 '1000092795.jpg#3\tA man in a blue shirt standing in a garden .',
 '1000092795.jpg#4\tTwo friends enjoy time spent together .',
 '10002456.jpg#0\tSeveral men in hard hats are operating a giant pulley system .',
 '10002456.jpg#1\tWorkers look down from up above on a piece of equipment .',
 '10002456.jpg#2\tTwo men working on a machine wearing hard hats .',
 '10002456.jpg#3\tFour men on top of a tall structure .',
 '10002456.jpg#4\tThree men on a large rig .']

In [20]:
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc:
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# parse descriptions
descriptions = load_descriptions(captions)
print('Loaded: %d ' % len(descriptions))

Loaded: 31783 


In [21]:
list(descriptions.keys())[:5]

['1000092795', '10002456', '1000268201', '1000344755', '1000366164']

In [22]:
descriptions['1000092795']

['Two young guys with shaggy hair look at their hands while hanging out in the yard .',
 'Two young , White males are outside near many bushes .',
 'Two men in green shirts are standing in a yard .',
 'A man in a blue shirt standing in a garden .',
 'Two friends enjoy time spent together .']

In [0]:
def clean_descriptions(desciptions):
  table = str.maketrans('', '', string.punctuation)
  for key, desc_list in desciptions.items():
    for i in range(len(desc_list)):
      desc = desc_list[i]
      desc = desc.split()
      desc = [word.lower() for word in desc]
      desc = [w.translate(table) for w in desc]
      desc = [word for word in desc if len(word)>1]
      desc = [word for word in desc if word.isalpha()]
      desc_list[i] =  ' '.join(desc)

clean_descriptions(descriptions)

In [24]:
descriptions['1000092795']

['two young guys with shaggy hair look at their hands while hanging out in the yard',
 'two young white males are outside near many bushes',
 'two men in green shirts are standing in yard',
 'man in blue shirt standing in garden',
 'two friends enjoy time spent together']

In [25]:
def to_vocabulary(descs):
  all_desc = set()

  for key in descs.keys():
    [all_desc.update(d.split()) for d in descs[key]]
  return all_desc

vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Original Vocabulary Size: 19735


In [0]:
# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, 'descriptions.txt')

In [0]:
save_descriptions(descriptions, '/content/drive/My Drive/Image_caption/descriptions.txt')

In [28]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc:
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

# load training dataset (6K)
filename = 'results_20130124.token'
train = load_set(filename)
print('Dataset: %d' % len(train))

Dataset: 31783


In [0]:
# Below path contains all the images
images = 'flickr30k-images/'
# Create a list of all image names in the directory
img = glob.glob(images + '*.jpg')

In [0]:
# Below file conatains the names of images to be used in train data
train_images_file = 'results_20130124.token'
# Read the train image names in a set
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Create a list of all the training images with their full path names
train_img = []

for i in img: # img is list of full path names of all images
    #if i[len(images):] in train_images: # Check if the image belongs to training set
    train_img.append(i) # Add it to the list of train images

In [31]:
len(train_img)

31783

In [32]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc:
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

# descriptions
train_descriptions = load_clean_descriptions('/content/drive/My Drive/descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

Descriptions: train=31783


In [0]:
def preprocess(image_path):
    # Convert all the images to size 299x299 as expected by the inception v3 model
    img = image.load_img(image_path, target_size=(299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [0]:
# Load the inception v3 model
model = InceptionV3(weights='imagenet')

In [0]:
# Create a new model, by removing the last layer (output layer) from the inception v3
model_new = Model(model.input, model.layers[-2].output)

In [0]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [0]:
# Call the funtion to encode all the train images
# This will take a while on CPU - Execute this only once
#start = time()
#encoding_train = {}
#for img in train_img:
#    encoding_train[img[len(images):]] = encode(img)
#print("Time taken in seconds = ", time()-start)

In [0]:
# Save the bottleneck train features to disk
#import pickle

#with open("encoded_train_images.pkl", "wb") as encoded_pickle:
#    pickle.dump(encoding_train, encoded_pickle)

In [0]:
#with open("/content/drive/My Drive/Image_caption/encoded_train_images.pkl", "wb") as encoded_pickle:
#    pickle.dump(encoding_train, encoded_pickle)

In [40]:
train_features = load(open("/content/drive/My Drive/Image_caption/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=31783


In [41]:
# Create a list of all the training captions
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
len(all_train_captions)

158915

In [42]:
# Consider only words which occur at least 10 times in the corpus
word_count_threshold = 5
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

preprocessed words 19737 -> 7664


In [0]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [44]:
vocab_size = len(ixtoword) + 1 # one for appended 0's
vocab_size

7665

In [0]:
import pickle

with open('wordtoix.pickle', 'wb') as handle:
    pickle.dump(wordtoix, handle, protocol=pickle.HIGHEST_PROTOCOL)


#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)

#print a == b

In [0]:
with open('ixtoword.pickle', 'wb') as handle:
    pickle.dump(ixtoword, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Description Length: 74


In [0]:

# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key+'.jpg']
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [0]:
#!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

#!unzip glove.6B.zip
#!rm glove.6B.zip

In [50]:
# Load Glove vectors

embeddings_index = {} # empty dictionary
f = open('/content/drive/My Drive/Image_caption/glove.6B.200d.txt', encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
embedding_dim = 200

# Get 200-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [52]:
embedding_matrix.shape

(7665, 200)

In [53]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)

outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [65]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 74)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 74, 200)      1533000     input_4[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_3[0][0]                    
____________________________________________________________________________________________

In [55]:
model.layers[2]

In [0]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [57]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
## TRYING OUT SAVED MODEL
from keras.models import load_model
new_model = load_model('/content/drive/My Drive/Image_caption/model/model_3.h5')

new_model.summary()



Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 74)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 74, 200)      1533000     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_2[0][0]                    
__________________________________________________________________________________________

In [0]:
epochs = 3
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

In [0]:
from keras.callbacks import *
filepath="/content/drive/My Drive/Image_caption/model/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
model = '/content/drive/My Drive/Image_caption/model/model_5.h5'
print(type(model))

<class 'str'>


In [51]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    new_model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    new_model.save('/content/drive/My Drive/Image_caption/model/new_model_' + str(i) + '.h5')

Epoch 1/1
10594/10594 [==============================] - 1699s 160ms/step - loss: 3.5403 - acc: 0.2993
Epoch 1/1
10594/10594 [==============================] - 1711s 162ms/step - loss: 3.5405 - acc: 0.2992
Epoch 1/1
10594/10594 [==============================] - 1734s 164ms/step - loss: 3.5407 - acc: 0.2992


In [0]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1, callbacks=callbacks_list)
    model.save('model_' + str(i) + '.h5')

In [0]:
model.optimizer.lr = 0.0001
epochs = 5
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [64]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('/content/drive/My Drive/Image_caption/model/f_new_model_' + str(i) + '.h5')

Epoch 1/1
5297/5297 [==============================] - 972s 183ms/step - loss: 3.4696 - acc: 0.3050


AttributeError: ignored

In [0]:
model.save_weights('model_30.h5')

In [0]:
model.load_weights('/content/drive/My Drive/Image_caption/model/new_model_2.h5')

In [0]:
test = '/content/drive/My Drive/Image_caption/test.jpg'

In [0]:
pic = encode(test)

In [0]:
pic = pic.reshape((1, 2048))

In [0]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [73]:
greedySearch(pic)

'dog is running through the grass'

In [0]:
test2 = '/content/test2.jpg'

In [76]:
pic = encode(test2)
pic = pic.reshape((1, 2048))
greedySearch(pic)

'man in blue shirt is riding his horse'